#### Purpose: Similarity of % variance: explore why?
First: Multivariate decompositions: Independent component analysis of fMRI
#### Author: Roza G. Bayrak
#### Derived from https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Clustering-Dimensionality-Reduction/Principal%20Component%20Analysis.ipynb

In [1]:
import nilearn as nil
import numpy as np
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from nilearn.image import iter_img
from nilearn.input_data import NiftiMasker, NiftiMapsMasker
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import TSNE
from nilearn.decomposition import CanICA
from nilearn import image, plotting
from nilearn.plotting import plot_stat_map, show, plot_prob_atlas

/home/bayrakrg/Tools/VENV/newest/lib/python3.6/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
folder = '/bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw'
all_scans = os.listdir(folder)
for scan in all_scans:
#     print(scan)
    pass
func_fname = os.path.join(folder, scan)

# Nifti Masker
masker = NiftiMasker(memory='nilearn_cache', memory_level=1, standardize=True)
data_masked = masker.fit_transform(func_fname)

In [ ]:
# Choose number of components
# rest = 84, task = 70 see more info on Glasser tICA paper
n_components = 84
ica = FastICA(n_components=n_components, random_state=0)
components_masked = ica.fit_transform(data_masked.T).T

In [ ]:
# Normalize estimated components, for thresholding to make sense
components_masked -= components_masked.mean(axis=0)
components_masked /= components_masked.std(axis=0)

# Threshold
components_masked[np.abs(components_masked) < .8] = 0

# Now invert the masking operation, going back to a full 3D
# representation
component_img = masker.inverse_transform(components_masked)

In [ ]:
# Show some interesting components
# Use the mean as a background
mean_img = image.mean_img(func_fname)
plot_stat_map(image.index_img(component_img, 0), mean_img)
plot_stat_map(image.index_img(component_img, 1), mean_img)
plot_stat_map(image.index_img(component_img, 2), mean_img)
plot_stat_map(image.index_img(component_img, 3), mean_img)
# plot_stat_map(image.index_img(component_img, 4), mean_img)
# plot_stat_map(image.index_img(component_img, 5), mean_img)
# plot_stat_map(image.index_img(component_img, 6), mean_img)
# plot_stat_map(image.index_img(component_img, 7), mean_img)
show()

### CanICA (Canonical ICA) - A group model for stable multi-subject ICA on fMRI datasets

i) probabilistic dimension reduction of the individual data, 

ii) canonical correlation analysis to identify a data subspace common to the group, 

iii) ICA-based pattern extraction.

In [ ]:
# Canonical Correlation Analysis runs after the PCA by default.

canica = CanICA(n_components=84, standardize=True, smoothing_fwhm=None,
                detrend=True, low_pass=0.01, high_pass=0.15, t_r=0.72,
                mask_strategy='background',
                memory="nilearn_cache", memory_level=1,
                verbose=10, n_jobs=2)

func_fnames = [folder + '/' + s for s in all_scans]
func_fnames = func_fnames[:500]
canica.fit(func_fnames)

[MultiNiftiMasker.fit] Loading data from [/bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_203923_rfMRI_REST1_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_195950_rfMRI_REST1_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_316835_rfMRI_REST2_RL_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_121921_rfMRI_REST2_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_715950_rfMRI_REST1_RL_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_286650_rfMRI_REST2_RL_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_700634_rfMRI_REST2_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_735148_rfMRI_REST1_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_896778_rfMRI_REST2_LR_blur4.nii.gz,
 /bigdata/HCP_rest/power+xifra/resting_minpro_blur/raw/fmri_134021_rfMRI_REST2_RL_blur4.nii.gz,

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:  1.3min
[Parallel(n_jobs=2)]: Done  14 tasks      | elapsed:  5.9min
[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed: 13.3min
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 22.3min
[Parallel(n_jobs=2)]: Done  68 tasks      | elapsed: 35.4min
[Parallel(n_jobs=2)]: Done  94 tasks      | elapsed: 51.9min
[Parallel(n_jobs=2)]: Done 124 tasks      | elapsed: 72.8min
[Parallel(n_jobs=2)]: Done 158 tasks      | elapsed: 100.3min
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed: 129.8min
[Parallel(n_jobs=2)]: Done 238 tasks      | elapsed: 166.3min
[Parallel(n_jobs=2)]: Done 284 tasks      | elapsed: 205.0min


In [ ]:
# Retrieve the independent components in brain space. Directly
# accessible through attribute `components_img_`.
canica_components_img = canica.components_img_

# Plot all ICA components together
plot_prob_atlas(canica_components_img, title='All ICA components')

In [ ]:
for i, cur_img in enumerate(iter_img(canica_components_img)):
    plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                  cut_coords=1, colorbar=False)

In [ ]:
masker = NiftiMapsMasker(maps_img=canica_components_img, standardize=True)
masker

In [ ]:
time_series = masker.fit_transform(func_fname)
print(time_series.shape)

In [ ]:
np.random.seed(77)

correlation_matrix = np.corrcoef(time_series.T)
fig = plt.figure(figsize=(10, 10), dpi=150)

# Mask out the major diagonal
np.fill_diagonal(correlation_matrix, 0)
labels=range(len(correlation_matrix))
plotting.plot_matrix(correlation_matrix, figure=fig, reorder=True, labels=list(labels), vmin=-0.8, vmax=0.8)

In [ ]:
for i, cur_img in enumerate(iter_img(canica_components_img)):
    if i == 28 or i == 51:
        plot_stat_map(cur_img, display_mode="z", title="IC %d" % i,
                      cut_coords=1, colorbar=False)

In [ ]:
plt.plot(time_series[:, 28])
plt.plot(time_series[:, 51])
plt.show()